In [2]:
import os
import subprocess
import time
import sys

print("checking environment...")

!pip install -U -q langchain langchain-community langchain-core langchain-ollama langgraph duckduckgo-search

print("Starting Ollama Server...")
if not os.path.exists("/usr/local/bin/ollama"):
    print("   -> Installing Ollama binary...")
    !curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen("ollama serve", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)
print("Checking/Downloading Llama 3 model...")
!ollama pull llama3

from typing import TypedDict
from langchain_ollama import ChatOllama
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.graph import StateGraph, END

print("\nInitializing Agents...")

try:
    llm = ChatOllama(model="llama3", temperature=0)
except Exception as e:
    print(f"Error connecting to LLM: {e}")
    sys.exit(1)

class AgentState(TypedDict):
    company_name: str
    raw_data: str
    final_report: str

def data_collector_node(state: AgentState):
    company = state['company_name']
    print(f"\nAgent 1: Searching for '{company}'...")

    try:
        search = DuckDuckGoSearchRun()
        results = search.invoke(f"{company} financial news 2024")
        print(f"   -> Search successful.")
    except Exception:
        print("   -> Search blocked/failed. Using simulation data.")
        results = f"Simulated financial data for {company}: Strong growth in 2024, stock up 15%."

    return {"raw_data": results}

def analyst_node(state: AgentState):
    print(f"Agent 2: Analyzing data...")
    data = state.get("raw_data")
    company = state.get("company_name")

    prompt = f"Analyze this data for {company}:\n{data}"
    response = llm.invoke(prompt)

    return {"final_report": response.content}

workflow = StateGraph(AgentState)
workflow.add_node("researcher", data_collector_node)
workflow.add_node("analyst", analyst_node)
workflow.set_entry_point("researcher")
workflow.add_edge("researcher", "analyst")
workflow.add_edge("analyst", END)
app = workflow.compile()

target_company = "Tesla"
print(f"--- Starting Analysis for: {target_company} ---")

inputs = {"company_name": target_company}
result = app.invoke(inputs)

print("\n" + "="*40)
print(f"FINAL REPORT: {target_company}")
print("="*40)
print(result['final_report'])

🔧 checking environment...
🚀 Starting Ollama Server...
⬇️ Checking/Downloading Llama 3 model...


🤖 Initializing Agents...
--- 🚀 Starting Analysis for: Tesla ---

🔎 Agent 1: Searching for 'Tesla'...
   -> Search blocked/failed. Using simulation data.
📝 Agent 2: Analyzing data...

FINAL REPORT: Tesla
A brief snippet of simulated financial data for Tesla! Let's dive into the analysis:

**Key Takeaway:** The data suggests that Tesla is expected to experience strong growth in 2024, with its stock price increasing by 15%.

**Growth Rate:** A 15% increase in stock price represents a significant growth rate. This could be attributed to various factors such as:

1. **Increasing demand for electric vehicles**: As the world shifts towards sustainable energy solutions, Tesla's market share and sales are likely to continue growing.
2. **Expansion into new markets**: Tesla's entry into new markets, such as China or Europe, could lead to increased revenue and profitability.
3. **Improving financial p